In [11]:
import numpy as np 
import pandas as pd
import os
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import shutil

In [12]:
for dirname, subdirs, filenames in os.walk('/kaggle/input'):
    print(dirname)


/kaggle/input
/kaggle/input/paddy-disease
/kaggle/input/paddy-disease/tungro
/kaggle/input/paddy-disease/yellow_stem_borer
/kaggle/input/paddy-disease/hispa
/kaggle/input/paddy-disease/downy_mildew
/kaggle/input/paddy-disease/black_stem_borer
/kaggle/input/paddy-disease/bacterial_leaf_streak
/kaggle/input/paddy-disease/bacterial_leaf_blight
/kaggle/input/paddy-disease/brown_spot
/kaggle/input/paddy-disease/blast
/kaggle/input/paddy-disease/leaf_roller
/kaggle/input/paddy-disease/normal
/kaggle/input/paddy-disease/bacterial_panicle_blight
/kaggle/input/paddy-disease/white_stem_borer


In [13]:
input_dir = '/kaggle/input/paddy-disease'
output_dir = '/kaggle/working/extracted_images'
os.makedirs(output_dir, exist_ok=True)

for dirname, _, filenames in os.walk(input_dir):
    if filenames:  
        image_name = filenames[0]  
        image_path = os.path.join(dirname, image_name)
       
       
        subdir_name = os.path.basename(dirname)
         
        try:
            with Image.open(image_path) as img:
                output_image_path = os.path.join(output_dir, f"{subdir_name}.jpg")
                img.save(output_image_path, dpi=(300, 300)) 
                print(f"Saved: {output_image_path} with 300 DPI")
        except Exception as e:
            print(f"Failed to process {image_path}: {e}")


shutil.make_archive('/kaggle/working/extracted_images', 'zip', output_dir)


Failed to process /kaggle/input/paddy-disease/metadata.csv: cannot identify image file '/kaggle/input/paddy-disease/metadata.csv'
Saved: /kaggle/working/extracted_images/tungro.jpg with 300 DPI
Saved: /kaggle/working/extracted_images/yellow_stem_borer.jpg with 300 DPI
Saved: /kaggle/working/extracted_images/hispa.jpg with 300 DPI
Saved: /kaggle/working/extracted_images/downy_mildew.jpg with 300 DPI
Saved: /kaggle/working/extracted_images/black_stem_borer.jpg with 300 DPI
Saved: /kaggle/working/extracted_images/bacterial_leaf_streak.jpg with 300 DPI
Saved: /kaggle/working/extracted_images/bacterial_leaf_blight.jpg with 300 DPI
Saved: /kaggle/working/extracted_images/brown_spot.jpg with 300 DPI
Saved: /kaggle/working/extracted_images/blast.jpg with 300 DPI
Saved: /kaggle/working/extracted_images/leaf_roller.jpg with 300 DPI
Saved: /kaggle/working/extracted_images/normal.jpg with 300 DPI
Saved: /kaggle/working/extracted_images/bacterial_panicle_blight.jpg with 300 DPI
Saved: /kaggle/worki

'/kaggle/working/extracted_images.zip'

In [ ]:
import tensorflow as tf
import os

input_dir = '/kaggle/input/paddy-disease'
BATCH_SIZE = 32  # Adjust according to TPU memory
IMAGE_SIZE = (224, 224)  # ResNet50 standard input size

# Map class names (subdirectory names) to integer labels
class_names = sorted([subdir for subdir in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, subdir))])
class_labels = {name: idx for idx, name in enumerate(class_names)}

# Load file paths and labels into lists
file_paths, labels = [], []
for class_name, label in class_labels.items():
    class_dir = os.path.join(input_dir, class_name)
    if os.path.isdir(class_dir):  # Check if it is a directory
        for filename in os.listdir(class_dir):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
                file_paths.append(os.path.join(class_dir, filename))
                labels.append(label)

# Create a dataset of file paths and labels
dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))

# Define functions for loading and augmenting images
def load_and_preprocess_image(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SIZE)
    img = img / 255.0  # Normalize to [0,1] range
    return img, label

def augment_image(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
    image = tf.image.random_hue(image, max_delta=0.2)
    return image, label

# Apply the load, preprocess, and augment functions
dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Set up TPU Strategy
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='') 
    tf.config.experimental_connect_to_cluster(resolver) 
    tf.tpu.experimental.initialize_tpu_system(resolver) 
    strategy = tf.distribute.TPUStrategy(resolver)
    print("TPU initialized")
except Exception as e:
    print("TPU could not be initialized:", e)
    strategy = None  # Set strategy to None if TPU initialization fails

# Define and compile the model under TPU scope if available
with (strategy.scope() if strategy else tf.distribute.get_strategy().scope()):
    model = tf.keras.applications.ResNet50(input_shape=(224, 224, 3), weights=None, classes=len(class_names))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(dataset, epochs=10)


TPU could not be initialized: Please provide a TPU Name to connect to.
Epoch 1/10


In [17]:
import os
import shutil

# Path to the working directory
working_dir = '/kaggle/working/'

# Loop through the working directory and delete all files and subdirectories
for filename in os.listdir(working_dir):
    file_path = os.path.join(working_dir, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  # Remove the file or symbolic link
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # Remove the directory
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')

print("All files and folders in '/kaggle/working/' have been deleted.")




All files and folders in '/kaggle/working/' have been deleted.
